# Tunability for DecisionTreeClassifier

# Intro

In [1]:
import pandas as pd

random_state = pd.read_csv("../../data/vars.csv")['random_state'].iloc[0]
n_iter = pd.read_csv("../../data/vars.csv")['n_iter'].iloc[0]
n_iter_BS = pd.read_csv("../../data/vars.csv")['n_iter_BS'].iloc[0]

In [2]:
df_college = pd.read_csv("../../data/processed/college.csv")
df_credit = pd.read_csv("../../data/processed/credit.csv")
df_diabetes = pd.read_csv("../../data/processed/diabetes.csv")
df_penguins = pd.read_csv("../../data/processed/penguins.csv")

In [3]:
print(df_college.info())
print()
print(df_diabetes.info())
print()
print(df_diabetes.info())
print()
print(df_penguins.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   type_school            1000 non-null   object 
 1   school_accreditation   1000 non-null   object 
 2   gender                 1000 non-null   object 
 3   interest               1000 non-null   object 
 4   residence              1000 non-null   object 
 5   parent_age             1000 non-null   int64  
 6   parent_salary          1000 non-null   int64  
 7   house_area             1000 non-null   float64
 8   average_grades         1000 non-null   float64
 9   parent_was_in_college  1000 non-null   bool   
 10  will_go_to_college     1000 non-null   int64  
dtypes: bool(1), float64(2), int64(3), object(5)
memory usage: 79.2+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 9 columns):
 #   Column                 Non-Null

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
import numpy as np


num_pipeline = Pipeline(steps=[
    ('num_impute', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('cat_impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OneHotEncoder(handle_unknown='ignore'))
])

col_trans = ColumnTransformer([
    ('num_pipeline', num_pipeline, make_column_selector(dtype_include = np.number)),
    ('cat_pipeline', cat_pipeline, make_column_selector(dtype_include = np.object_))
])

model_pipeline = Pipeline(steps=[
    ('preprocessing', col_trans),
    ('model', DecisionTreeClassifier())
])

# RandomizedSearchCV

In [5]:
tree_params = {
    'model__criterion' : ['gini', 'entropy', 'log_loss'],
    'model__max_depth': range(1, 30), # paper
    # 'model__min_samples_split': range(2, 21),                  
    # 'model__min_samples_leaf': range(1, 21),                         
    'model__max_leaf_nodes': range(10, 100, 10)
}

## Credit dataset

In [6]:
from sklearn.model_selection import RandomizedSearchCV

X = df_credit.iloc[:, 0:-1]
y = df_credit.iloc[:, -1]

random_search_model = RandomizedSearchCV(estimator = model_pipeline,
                                   param_distributions = tree_params,
                                   n_iter = n_iter,
                                   cv = 5,
                                   scoring = 'f1',
                                   random_state = random_state)

random_search_model.fit(X, y)

best_f1_credit = random_search_model.best_score_

print(f"Best f1 score: {best_f1_credit}")
print(f"Best params: {random_search_model.best_params_}")

random_search_model.best_estimator_

Best f1 score: 0.8648002281347171
Best params: {'model__max_leaf_nodes': 40, 'model__max_depth': 2, 'model__criterion': 'log_loss'}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000019E4ACE0F50>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000019E48CEC9D0>)])),
                ('model',
                 DecisionTreeClassifier(criterion='log_loss', max_depth=2,
                                        max_leaf_nodes=40))])

## Data frame with params, results and mean result

In [7]:
tested_params = random_search_model.cv_results_['params']

In [8]:
df_decision_tree = pd.DataFrame(tested_params)
df_decision_tree['credit_f1'] = random_search_model.cv_results_['mean_test_score']
df_decision_tree

,model__max_leaf_nodes,model__max_depth,model__criterion,credit_f1
0,70,4,gini,0.820041
1,20,18,log_loss,0.813850
2,80,19,gini,0.775081
3,60,5,entropy,0.805339
4,40,2,log_loss,0.864800
...,...,...,...,...
195,10,14,log_loss,0.809464
196,50,25,log_loss,0.805688
197,50,24,gini,0.804764
198,50,26,log_loss,0.797036


## College dataset

In [9]:
from sklearn.model_selection import GridSearchCV

X = df_college.iloc[:, 0:-1]
y = df_college.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_decision_tree['college_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_college = grid_search_model.best_score_

print(f"Best f1 score: {best_f1_college}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

Best f1 score: 0.8554041485164052
Best params: {'model__criterion': 'gini', 'model__max_depth': 23, 'model__max_leaf_nodes': 20}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000019E4AF4DE90>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000019E4AF7F7D0>)])),
                ('model',
                 DecisionTreeClassifier(max_depth=23, max_leaf_nodes=20))])

## Diabetes dataset

In [10]:
from sklearn.model_selection import GridSearchCV

X = df_diabetes.iloc[:, 0:-1]
y = df_diabetes.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_decision_tree['diabetes_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_diabetes = grid_search_model.best_score_

print(f"Best f1 score: {best_f1_diabetes}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

Best f1 score: 0.6330524416609065
Best params: {'model__criterion': 'gini', 'model__max_depth': 28, 'model__max_leaf_nodes': 10}


c:\Users\natal\.conda\envs\tunability\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000019E4AF5BD10>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000019E4A30D490>)])),
                ('model',
                 DecisionTreeClassifier(max_depth=28, max_leaf_nodes=10))])

## Penguins dataset

In [11]:
from sklearn.model_selection import GridSearchCV

X = df_penguins.iloc[:, 0:-1]
y = df_penguins.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_decision_tree['penguins_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_penguins = grid_search_model.best_score_


print(f"Best f1 score: {best_f1_penguins}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

Best f1 score: 0.9967213114754099
Best params: {'model__criterion': 'entropy', 'model__max_depth': 6, 'model__max_leaf_nodes': 40}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000019E4AF21E10>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000019E4AF6C890>)])),
                ('model',
                 DecisionTreeClassifier(criterion='entropy', max_depth=6,
                                        max_leaf_nodes=40))])

## Tunability 

In [12]:
df_decision_tree['mean_f1'] = df_decision_tree[['credit_f1', 'college_f1', 'diabetes_f1', 'penguins_f1']].mean(axis=1)
df_decision_tree

,model__max_leaf_nodes,model__max_depth,model__criterion,credit_f1,college_f1,diabetes_f1,penguins_f1,mean_f1
0,70,4,gini,0.820041,0.832752,0.617536,0.993331,0.815915
1,20,18,log_loss,0.813850,0.843925,0.604201,0.989944,0.812980
2,80,19,gini,0.775081,0.827601,0.568130,0.993333,0.791036
3,60,5,entropy,0.805339,0.850968,0.611861,0.993331,0.815375
4,40,2,log_loss,0.864800,0.729040,0.546090,0.993331,0.783316
...,...,...,...,...,...,...,...,...
195,10,14,log_loss,0.809464,0.844462,0.620412,0.989944,0.816070
196,50,25,log_loss,0.805688,0.833628,0.532898,0.993333,0.791387
197,50,24,gini,0.804764,0.839340,0.579773,0.993331,0.804302
198,50,26,log_loss,0.797036,0.835120,0.549574,0.993333,0.793766


In [13]:
# sigma star
sigma_star_row = df_decision_tree[df_decision_tree['mean_f1'] == max(df_decision_tree['mean_f1'])]
sigma_star_row

,model__max_leaf_nodes,model__max_depth,model__criterion,credit_f1,college_f1,diabetes_f1,penguins_f1,mean_f1
39,10,26,gini,0.839449,0.848145,0.633052,0.996721,0.829342
184,10,16,gini,0.839449,0.848145,0.633052,0.996721,0.829342


### (3.3) Measuring Overall Tunability of a ML Algorithm

In [14]:
d_credit = -sigma_star_row['credit_f1'].iloc[0] + best_f1_credit
d_college = -sigma_star_row['college_f1'].iloc[0] + best_f1_college
d_diabetes = -sigma_star_row['diabetes_f1'].iloc[0] + best_f1_diabetes
d_penguins = -sigma_star_row['penguins_f1'].iloc[0] + best_f1_penguins

d = pd.Series([d_credit, d_college, d_diabetes, d_penguins])
print(d)
print(f"Mean: {d.mean()}")


0    0.025352
1    0.007259
2    0.000000
3    0.000000
dtype: float64
Mean: 0.008152655020469773


### Saving results

In [15]:
df_decision_tree.to_csv("../../data/RSmodels/dt_results.csv", index=False)

# BayesSearchCV

In [15]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

n_iter_BS = pd.read_csv('../../data/vars.csv')['n_iter_BS'].iloc[0]
BS_iterations = pd.read_csv('../../data/vars.csv')['BS_iterations'].iloc[0]

tree_params = {
    'model__criterion' : Categorical(['gini', 'entropy', 'log_loss']),
    'model__max_depth': Integer(1, 30),
    # 'model__min_samples_split': Integer(2, 21),                  
    # 'model__min_samples_leaf': Integer(1, 21),                         
    'model__max_leaf_nodes': Integer(10, 100)
}

## Credit dataset

In [16]:
X = df_credit.iloc[:, 0:-1]
y = df_credit.iloc[:, -1]


bayes_search_credit_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=tree_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_credit_model.fit(X, y)

best_BS_f1_credit = bayes_search_credit_model.best_score_
credit_best_estimator = bayes_search_credit_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_credit}")

credit_best_estimator

c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 2, 68] before, using random point ['entropy', 28, 87]
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 2, 64] before, using random point ['gini', 9, 46]
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 2, 64] before, using random point ['log_loss', 11, 10]
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _dat

Best f1 score: 0.8648002281347171


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000016EDA57E710>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000016EDA56AF90>)])),
                ('model',
                 DecisionTreeClassifier(criterion='entropy', max_depth=2,
                                        max_leaf_nodes=17))])

## College dataset

In [17]:
X = df_college.iloc[:, 0:-1]
y = df_college.iloc[:, -1]


bayes_search_college_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=tree_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_college_model.fit(X, y)

best_BS_f1_college = bayes_search_college_model.best_score_
college_best_estimator = bayes_search_college_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_college}")

college_best_estimator

c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 5, 100] before, using random point ['log_loss', 22, 77]
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 5, 100] before, using random point ['entropy', 11, 56]
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 23, 10] before, using random point ['log_loss', 11, 46]
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 5, 100] before, using random point ['entropy', 23, 85]
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimiz

Best f1 score: 0.8572646136326844


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000016ED9D82110>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000016ED9D815D0>)])),
                ('model',
                 DecisionTreeClassifier(max_depth=12, max_leaf_nodes=20))])

## Diabetes dataset

In [18]:
X = df_diabetes.iloc[:, 0:-1]
y = df_diabetes.iloc[:, -1]


bayes_search_diabetes_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=tree_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_diabetes_model.fit(X, y)

best_BS_f1_diabetes = bayes_search_diabetes_model.best_score_
diabetes_best_estimator = bayes_search_diabetes_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_diabetes}")
diabetes_best_estimator

c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 20, 10] before, using random point ['log_loss', 13, 92]
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 20, 10] before, using random point ['entropy', 3, 93]
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 15, 10] before, using random point ['entropy', 5, 41]
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 20, 10] before, using random point ['log_loss', 8, 25]
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.

Best f1 score: 0.6470280233641607


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000016ED9D2CE90>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000016EDC745290>)])),
                ('model',
                 DecisionTreeClassifier(max_depth=18, max_leaf_nodes=17))])

## Penguins dataset

In [19]:
X = df_penguins.iloc[:, 0:-1]
y = df_penguins.iloc[:, -1]


bayes_search_penguins_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=tree_params,
                                n_iter=int(n_iter_BS),
                                #random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_penguins_model.fit(X, y)

best_BS_f1_penguins = bayes_search_penguins_model.best_score_
penguins_best_estimator = bayes_search_penguins_model.best_estimator_

print(f"Best f1 score: {best_BS_f1_penguins}")

penguins_best_estimator

c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 23, 10] before, using random point ['entropy', 2, 93]
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['log_loss', 25, 100] before, using random point ['gini', 21, 13]
  warnings.warn(
c:\Users\flang\anaconda3\envs\tunability\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['gini', 22, 10] before, using 

Best f1 score: 0.9967213114754099


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000016EDC6C5150>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000016EDC7F73D0>)])),
                ('model',
                 DecisionTreeClassifier(max_depth=9, max_leaf_nodes=53))])

## Tunability

### (3.3) Measuring Overall Tunability of a ML Algorithm

In [20]:
d_BS_credit = -sigma_star_row['credit_f1'].iloc[0] + best_BS_f1_credit
d_BS_college = -sigma_star_row['college_f1'].iloc[0] + best_BS_f1_college
d_BS_diabetes = -sigma_star_row['diabetes_f1'].iloc[0] + best_BS_f1_diabetes
d_BS_penguins = -sigma_star_row['penguins_f1'].iloc[0] + best_BS_f1_penguins

d_BS = pd.Series([d_BS_credit, d_BS_college, d_BS_diabetes, d_BS_penguins])
print(d_BS)
print(f"Mean: {d_BS.mean()}")

0    0.025352
1    0.009119
2    0.013976
3    0.003390
dtype: float64
Mean: 0.012959124352471735


### Models saving

In [22]:
from joblib import dump

dump(bayes_search_college_model, '../../data/BSmodels/DecisionTree/bayes_search_college_model.joblib')
dump(bayes_search_credit_model, '../../data/BSmodels/DecisionTree/bayes_search_credit_model.joblib')
dump(bayes_search_diabetes_model, '../../data/BSmodels/DecisionTree/bayes_search_diabetes_model.joblib')
dump(bayes_search_penguins_model, '../../data/BSmodels/DecisionTree/bayes_search_penguins_model.joblib')

['../../data/BSmodels/DecisionTree/bayes_search_penguins_model.joblib']